**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity 
BASE DIRECTORY (FD_BASE): /mount 
REPO DIRECTORY (FD_REPO): /mount/repo 
WORK DIRECTORY (FD_WORK): /mount/work 
DATA DIRECTORY (FD_DATA): /mount/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /mount/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /mount/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /mount/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /mount/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /mount/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /mount/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /mount/repo/Proj_ENCODE_FCC/log 
PROJECT APP     (FD_APP): /mount/repo/Proj_ENCODE_FCC/app 
PROJECT REF     (FD_REF): /mount/repo/Proj_ENCODE_FCC/references 



**Set global variables**

In [2]:
TXT_FOLDER_REGION = "hic_insitu_K562_ENCSR545YBD"
TXT_ASSAY = "hic_insitu"
TXT_INDEX = "ENCSR545YBD"

## Import data

In [3]:
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_DAT, "external", txt_folder)

vec = dir(txt_fdiry)
for (txt in vec){cat(txt, "\n")}

K562.hg38.ENCSR545YBD.ENCFF271SAF.hic_insitu.contact_domain.bedpe.gz 
K562.hg38.ENCSR545YBD.ENCFF616PUW.hic_insitu.matrix.hic 
K562.hg38.ENCSR545YBD.ENCFF693XIL.hic_insitu.loop.bedpe.gz 
run_download.log.txt 
run_download.sh 


In [4]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_DAT, "external", txt_folder)
txt_fname = "K562.hg38.ENCSR545YBD.ENCFF271SAF.hic_insitu.contact_domain.bedpe.gz"
txt_fpath = file.path(txt_fdiry, txt_fname)

### read table
dat = read_tsv(txt_fpath, show_col_types = FALSE)

### arrange columns
dat = dat %>% dplyr::filter(!row_number() == 1)
colnames(dat)[1] = "chr"

### assign and show
dat_region_import = dat
print(dim(dat))
fun_display_table(head(dat))

New names:
• `score` -> `score...8`
• `score` -> `score...12`
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details, e.g.:
  dat <- vroom(...)
  problems(dat)”


[1] 5703   16


chr,x1,x2,chr2,y1,y2,name,score...8,strand1,strand2,color,score...12,uVarScore,lVarScore,upSign,loSign
chr10,84565000,86085000,chr10,84565000,86085000,.,.,.,.,2552550,0.7973227,0.3603668,0.3402993,0.4446594,0.5478586
chr10,50985000,52225000,chr10,50985000,52225000,.,.,.,.,2552550,1.0083018,0.3171058,0.3477260,0.5433548,0.4545161
chr10,98415000,99300000,chr10,98415000,99300000,.,.,.,.,2552550,0.6516994,0.2746902,0.2858612,0.4402222,0.6153263
chr10,112500000,113595000,chr10,112500000,113595000,.,.,.,.,2552550,1.1134926,0.2805900,0.2788020,0.6329752,0.5778512
chr10,98715000,99265000,chr10,98715000,99265000,.,.,.,.,2552550,0.6108739,0.2388994,0.2776476,0.4019481,0.4756494
chr10,94630000,95075000,chr10,94630000,95075000,.,.,.,.,2552550,1.0027722,0.2445123,0.2682168,0.6469136,0.4656790


In [5]:
colnames(dat)

[1] "chr"        "x1"         "x2"         "chr2"       "y1"        
 [6] "y2"         "name"       "score...8"  "strand1"    "strand2"   
[11] "color"      "score...12" "uVarScore"  "lVarScore"  "upSign"    
[16] "loSign"

## Arrange loops

In [6]:
### init
dat = dat_region_import
vec_txt_cname1 = c(
    "chr",  "x1", "x2", 
    "chr2", "y1", "y2")
vec_txt_cname2 = c(
    "Chrom_A", "ChromStart_A", "ChromEnd_A",
    "Chrom_B", "ChromStart_B", "ChromEnd_B"
)
vec_txt_cname3 = c(
    "Chrom_A", "ChromStart_A", "ChromEnd_A", "Region_A", "Length_A",
    "Chrom_B", "ChromStart_B", "ChromEnd_B", "Region_B", "Length_B",
    "Name", "Size"
)

### get columns
dat = dat %>% dplyr::select(!!!vec_txt_cname1)
colnames(dat) = vec_txt_cname2

### arrange columns
dat = dat %>% 
    dplyr::mutate(
        Length_A = ChromEnd_A - ChromStart_A,
        Length_B = ChromEnd_B - ChromStart_B,
        Size     = (Length_A + Length_B) / 2,
        Region_A   = paste0(Chrom_A, ":", ChromStart_A, "-", ChromEnd_A),
        Region_B   = paste0(Chrom_B, ":", ChromStart_B, "-", ChromEnd_B),
        Name       = paste0(Region_A, "|", Region_B)
    ) %>%
    dplyr::select(!!!vec_txt_cname3)

### assign and show
dat_region_arrange = dat
print(dim(dat))
fun_display_table(head(dat))

[1] 5703   12


Chrom_A,ChromStart_A,ChromEnd_A,Region_A,Length_A,Chrom_B,ChromStart_B,ChromEnd_B,Region_B,Length_B,Name,Size
chr10,84565000,86085000,chr10:84565000-86085000,1520000,chr10,84565000,86085000,chr10:84565000-86085000,1520000,chr10:84565000-86085000|chr10:84565000-86085000,1520000
chr10,50985000,52225000,chr10:50985000-52225000,1240000,chr10,50985000,52225000,chr10:50985000-52225000,1240000,chr10:50985000-52225000|chr10:50985000-52225000,1240000
chr10,98415000,99300000,chr10:98415000-99300000,885000,chr10,98415000,99300000,chr10:98415000-99300000,885000,chr10:98415000-99300000|chr10:98415000-99300000,885000
chr10,112500000,113595000,chr10:112500000-113595000,1095000,chr10,112500000,113595000,chr10:112500000-113595000,1095000,chr10:112500000-113595000|chr10:112500000-113595000,1095000
chr10,98715000,99265000,chr10:98715000-99265000,550000,chr10,98715000,99265000,chr10:98715000-99265000,550000,chr10:98715000-99265000|chr10:98715000-99265000,550000
chr10,94630000,95075000,chr10:94630000-95075000,445000,chr10,94630000,95075000,chr10:94630000-95075000,445000,chr10:94630000-95075000|chr10:94630000-95075000,445000


**Check**

In [7]:
dat  = dat_region_arrange

vec1 = dat$Region_A
vec2 = dat$Region_B
print(all(vec1 == vec2))

vec1 = dat$Length_A
vec2 = dat$Length_B
print(all(vec1 == vec2))

[1] TRUE
[1] TRUE


## Prepare region

In [8]:
### init
vec_txt_cname_A = c("Chrom_A", "ChromStart_A", "ChromEnd_A", "Region_A")
vec_txt_cname   = c("Chrom",   "ChromStart",   "ChromEnd",   "Region", "Group", "Label")
txt_assay = TXT_ASSAY

### prepare region
dat = dat_region_arrange
dat = dat %>% 
    dplyr::select(!!!vec_txt_cname_A) %>% 
    dplyr::mutate(
        Group = txt_assay,
        Label = "TAD"
    ) %>%
    dplyr::distinct()
colnames(dat) = vec_txt_cname

### assign and show
dat_region_simplify = dat
print(dim(dat))
fun_display_table(head(dat))

[1] 5703    6


Chrom,ChromStart,ChromEnd,Region,Group,Label
chr10,84565000,86085000,chr10:84565000-86085000,hic_insitu,TAD
chr10,50985000,52225000,chr10:50985000-52225000,hic_insitu,TAD
chr10,98415000,99300000,chr10:98415000-99300000,hic_insitu,TAD
chr10,112500000,113595000,chr10:112500000-113595000,hic_insitu,TAD
chr10,98715000,99265000,chr10:98715000-99265000,hic_insitu,TAD
chr10,94630000,95075000,chr10:94630000-95075000,hic_insitu,TAD


## Save results

In [9]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder)
txt_fname = "K562.hg38.hic_insitu.TAD.bedpe.gz"
txt_fpath = file.path(txt_fdiry, txt_fname)

### get table
dat = dat_region_simplify
dat = dat %>% dplyr::arrange(Chrom, ChromStart, ChromEnd)

### write tabel
dir.create(txt_fdiry, showWarnings = FALSE)
write_tsv(dat, txt_fpath, col_names = FALSE)

In [10]:
### set file path
txt_folder = TXT_FOLDER_REGION
txt_fdiry  = file.path(FD_RES, "region", txt_folder, "summary")
txt_fname = "K562.hg38.hic_insitu.TAD.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

### get table
dat = dat_region_arrange
dat = dat %>% dplyr::arrange(
    Chrom_A, ChromStart_A, ChromEnd_A,
    Chrom_B, ChromStart_B, ChromEnd_B
)

### write tabel
dir.create(txt_fdiry, showWarnings = FALSE)
write_tsv(dat, txt_fpath)